In [1]:
import os,sys,json
from glob import glob

os.chdir('/home/avijit/projects/Awareness_vs_Unawareness')
sys.path.insert(0,"/home/avijit/projects/Awareness_vs_Unawareness/robustfairnesscode")

In [2]:
import warnings
warnings.filterwarnings('ignore')

from robustfairnesscode import data, losses, optimization, model, utils, dro_training, softweights_training
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
import traceback

In [4]:
def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr,f)
    f.close()

In [5]:
path = 'data/datasets/credit_tw/'
synthfols = glob("data/datasets/credit_tw/synthetic/*/")
paths = [path]
paths.extend(synthfols)

In [6]:
def task(path):
    
    print(path)

    if True:#os.path.exists(path+'preds/Softweights_pred.json') == False:
        
        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'DEFAULT_PAY']


#         train_df.index = train_df['SEX']

#         train_df['SEX'] = train_df['SEX'].map({'Male': 1, 'Female': 0})


        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'DEFAULT_PAY']



#         test_df.index = test_df['SEX']
        
#         train_df['SEX'] = train_df['SEX'].map({'Male': 1, 'Female': 0})

        

        X_train = train_df.drop(['DEFAULT_PAY'],axis=1)
        X_test = test_df.drop(['DEFAULT_PAY'],axis=1)

        Y_train = pd.Series(train_df['DEFAULT_PAY'])
        Y_test = pd.Series(test_df['DEFAULT_PAY'])

        Y_train = pd.Series(Y_train.factorize(sort=True)[0], index=Y_train.index)
        Y_test = pd.Series(Y_test.factorize(sort=True)[0], index=Y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        train_df  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        test_df = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)
        
        train_df['DEFAULT_PAY'] = Y_train
        test_df['DEFAULT_PAY'] = Y_test

        print(train_df.columns)

        LABEL_COLUMN = "DEFAULT_PAY"
        FEATURE_NAMES = list(train_df.columns)
        FEATURE_NAMES.remove(LABEL_COLUMN)
        PROTECTED_COLUMNS = ['SEX_Female','SEX_Male']

        PROXY_COLUMNS = PROTECTED_COLUMNS 


        val,_,_,_ = dro_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES, 
                                                          PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 1,constraint='tpr_and_fpr')

        vall = [float(v) for v in val[0]]

        val_m = [0 if i < 0 else 1 for i in vall]

        writejson(path,val_m,'DRO_pred.json')

        val2,_,_,_ = softweights_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES, 
                                                                   PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 1, constraint='tpr_and_fpr')

        val2l = [float(v) for v in val2[0]]

        val_m2 = [0 if i < 0 else 1 for i in val2l]

        writejson(path,val_m2,'Softweights_pred.json')

        return path, 'DRO and SW done'

In [ ]:
import random
random.shuffle(paths)
for path in paths:
    try:
        task(path)
    except:
        traceback.print_exc()

data/datasets/credit_tw/synthetic/gender_flip_labels0.7_version4/
Index(['SEX_Female', 'SEX_Male', 'LIMIT_BAL', 'EDUCATION', 'MARRIAGE', 'AGE',
       'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'DEFAULT_PAY'],
      dtype='object')
Split 1 of 1
time since start: 0.038840532302856445
begin optimizing learning rate p list: [0.001, 0.001, 0.001]
begin optimizing learning rate theta: 0.001 learning rate lambda: 0.100
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


minibatch is off

start loop  1 in loops  1
finish loop  1 in loops  1
time for this loop  1.6201260089874268
tpr_overall 0.9

In [1]:
# from multiprocessing.pool import ThreadPool as Pool

# pool = Pool(10)
# for result in pool.imap_unordered(task, paths):
#     print(result)

In [ ]:
#   Args:
#     predictions: numpy array of floats representing predictions. Predictions are treated 
#       as positive classification if value is >0, and negative classification if value is <= 0.
#     labels: numpy array of floats representing labels. labels are also treated as positive 
#       classification if value is >0, and negative classification if value is <= 0.